In [4]:
import json
import plotly.graph_objects as go

from src.core import file_manager as fm
from src.embeddings.constants import EMBEDDING_MODELS_TRANSLATION

In [5]:
VARIATIONS_TRANSLATE = {
    'k100': 'Todas as sentenças',
    'k100_without_outliers': 'Sem outliers',
    'k100_without_sentences_higher_than_median': 'Distância até a Mediana'
}


def get_results(base_dir):
    results = {}

    for variation in VARIATIONS_TRANSLATE.keys():
        variation_translated = VARIATIONS_TRANSLATE[variation]
        results[variation_translated] = {}

        for embedding in EMBEDDING_MODELS_TRANSLATION.keys():
            filename = f'{base_dir}/{variation}/{embedding}/results/intent_report.json'

            intent_report = json.load(open(filename))

            embedding_translated = EMBEDDING_MODELS_TRANSLATION[embedding]

            results[variation_translated][embedding_translated] = intent_report['accuracy']

    return results


def plot_chart(results):
    data = [
        go.Bar(
            name=variation,
            x=list(results[variation].keys()),
            y=list(results[variation].values()),
            marker_color=color
        )

        for variation, color in zip(results.keys(), ['DarkGoldenRod', 'DarkOliveGreen', 'DarkSlateBlue'])
    ]

    fig = go.Figure(data=data, layout_yaxis_range=[0, 1])

    # Change the bar mode
    fig.update_layout(barmode='group',  xaxis_tickangle=-30)
    fig.show()


def run_pipeline(path):
    results = get_results(path)

    plot_chart(results)


In [6]:
results = get_results(fm.filename_from_data_dir('nlu_models/patient'))

results_without_others_intent = get_results(fm.filename_from_data_dir('nlu_models/patient/without_others_intent'))

In [7]:
import pandas as pd

pd.DataFrame(results)

,Todas as sentenças,Sem outliers,Distância até a Mediana
BERTimbau,0.784900,0.796314,0.813875
FLAIR,0.804523,0.795429,0.832152
Glove,0.798542,0.817856,0.831852
LaBSE,0.834610,0.851592,0.891791
MUSE,0.874416,0.880453,0.917910


In [8]:
pd.DataFrame(results_without_others_intent)

,Todas as sentenças,Sem outliers,Distância até a Mediana
BERTimbau,0.892290,0.877239,0.927036
FLAIR,0.919286,0.917555,0.927602
Glove,0.934948,0.945182,0.960674
LaBSE,0.917761,0.920836,0.961923
MUSE,0.943280,0.943182,0.971861


In [9]:
run_pipeline(fm.filename_from_data_dir('nlu_models/patient'))

In [10]:
run_pipeline(fm.filename_from_data_dir('nlu_models/patient/without_others_intent'))

In [11]:
import copy

results_merged = copy.deepcopy(results)

for variation in results_without_others_intent.keys():
  for embedding_key in results_without_others_intent[variation].keys():
    new_key = f'{embedding_key} ~ (others)'
    results_merged[variation][new_key] = results_without_others_intent[variation][embedding_key]
    
plot_chart(results_merged)